# 2. Finnhub API 기반 뉴스 데이터 전처리 및 FinBERT 감정분석

### 2-1. 파일 경로 설정
분석할 뉴스 데이터 파일의 읽기 경로와 감정분석 결과를 저장할 경로를 설정합니다.


In [23]:
# 입력 파일 경로: Finnhub에서 수집한 Apple 뉴스 데이터 CSV 파일
read_path = "./AAPL_extended_news_2025-06-14.csv"

# 출력 파일 경로: FinBERT 감정분석 결과가 추가된 CSV 파일
write_path = "./apple_finbert_finnhub.csv"

### 2-2. 데이터 로드
pandas를 사용하여 Finnhub에서 수집한 뉴스 데이터를 DataFrame으로 로드합니다.


In [24]:
# 데이터 분석을 위한 pandas 라이브러리 import
import pandas as pd

# CSV 파일에서 뉴스 데이터를 DataFrame으로 로드
# 파일에는 id, title, summary, link, publisher, pubDate 등의 컬럼이 포함됨
df = pd.read_csv(read_path)

### 2-3. FinBERT 모델 설정 및 데이터 전처리
금융 도메인에 특화된 FinBERT 모델을 로드하고, 뉴스 텍스트를 감정분석에 적합하게 전처리합니다.


In [27]:
# 감정분석을 위한 transformers 라이브러리 import
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# FinBERT 모델 설정 - 금융 뉴스 감정분석에 특화된 사전훈련 모델
MODEL = "yiyanghkust/finbert-tone"  # Hugging Face의 FinBERT-tone 모델

# 토크나이저 로드 - 텍스트를 토큰으로 변환하는 도구
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# 사전훈련된 FinBERT 모델 로드 - 금융 텍스트의 감정을 분류하는 모델
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# 감정분석 파이프라인 생성
# return_all_scores=True: positive, neutral, negative 모든 점수를 반환
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

# 이전에 로드한 뉴스 데이터 사용
df = pd.read_csv(read_path)

# 빈 값(NaN)을 빈 문자열로 대체하여 처리 오류 방지
df = df.fillna("")

# 분석할 텍스트 컬럼 생성 - 뉴스 제목을 감정분석 대상으로 설정
# title 컬럼의 내용을 text 컬럼으로 복사
df["text"] = df["title"] 

# 텍스트 길이 제한 및 플래그 함수 정의
def truncate_and_flag(text, tokenizer, max_len=512):
    """
    텍스트를 BERT 모델의 최대 토큰 길이에 맞게 자르고, 
    원본이 길이 제한을 초과했는지 플래그로 표시하는 함수
    
    Args:
        text: 처리할 텍스트
        tokenizer: 사용할 토크나이저
        max_len: 최대 토큰 길이 (BERT 계열은 보통 512)
        
    Returns:
        tuple: (잘린_텍스트, 초과_플래그)
    """
    # 텍스트를 토큰으로 변환하면서 max_len에 맞게 자르기
    tokens = tokenizer.encode(text, truncation=True, max_length=max_len)
    
    # 잘린 토큰을 다시 텍스트로 변환
    truncated_text = tokenizer.decode(tokens, skip_special_tokens=True)
    
    # 원본 텍스트가 최대 길이를 초과했는지 확인
    # 1: 초과함, 0: 초과하지 않음
    over_flag = 1 if len(tokenizer.encode(text)) > max_len else 0
    
    return truncated_text, over_flag

# 모든 텍스트에 대해 길이 제한 적용 및 초과 플래그 생성
# apply()와 pd.Series()를 사용하여 함수 결과를 두 개의 컬럼으로 분할
df[["text", "over_512"]] = df["text"].apply(
    lambda x: pd.Series(truncate_and_flag(x, tokenizer, max_len=512))
)


Device set to use cuda:0
c:\Users\jhh33\anaconda3\envs\pythonProject1\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


결과 저장 완료: ./apple_finbert_finnhub.csv


### 2-4. FinBERT 감정분석 실행
전처리된 뉴스 텍스트에 대해 FinBERT 모델을 사용하여 감정분석을 수행하고 결과를 저장합니다.


In [ ]:
# FinBERT 감정분석 실행을 위한 텍스트 리스트 준비
# DataFrame의 text 컬럼을 파이썬 리스트로 변환
texts = df["text"].tolist()

# FinBERT 모델로 배치 감정분석 수행
# batch_size=8: 한 번에 8개씩 처리하여 메모리 효율성과 속도 최적화
# GPU 메모리 부족시 더 작은 값(예: 4, 2)으로 조정 가능
results = finbert(texts, batch_size=8)

# 감정분석 결과를 저장할 빈 리스트들 초기화
pos_scores, neu_scores, neg_scores = [], [], []

# 각 텍스트의 감정분석 결과를 순회하며 점수 추출
for res in results:
    # 결과를 라벨:점수 형태의 딕셔너리로 변환
    # 라벨을 소문자로 통일하여 일관성 확보
    d = {r["label"].lower(): r["score"] for r in res}
    
    # 각 감정에 대한 점수를 리스트에 추가 (없으면 0.0으로 기본값 설정)
    pos_scores.append(d.get("positive", 0.0))   # 긍정 감정 점수
    neu_scores.append(d.get("neutral", 0.0))    # 중립 감정 점수  
    neg_scores.append(d.get("negative", 0.0))   # 부정 감정 점수

# 감정분석 결과를 DataFrame에 새로운 컬럼으로 추가
df["finbert_positive"] = pos_scores   # FinBERT 긍정 점수 (0~1)
df["finbert_neutral"] = neu_scores    # FinBERT 중립 점수 (0~1)
df["finbert_negative"] = neg_scores   # FinBERT 부정 점수 (0~1)

# 감정분석 결과가 추가된 DataFrame을 CSV 파일로 저장
# index=False: 행 번호를 파일에 포함하지 않음
df.to_csv(write_path, index=False)

# 저장 완료 메시지 출력
print(f"결과 저장 완료: {write_path}")


2. Text의 길이가 얼마나 영향을 끼치는지 확인

In [13]:
df = pd.read_csv("./apple_finbert_finnhub.csv")

# 512 토큰 넘은 것만 필터링
df_over = df[df["over_512"] == 1]

# 요약 통계 출력
summary = {
    "총 개수": len(df_over),
    "평균 Positive": df_over["finbert_positive"].mean(),
    "평균 Neutral": df_over["finbert_neutral"].mean(),
    "평균 Negative": df_over["finbert_negative"].mean(),
    "Positive > 0.5": (df_over["finbert_positive"] > 0.5).sum(),
    "Negative > 0.5": (df_over["finbert_negative"] > 0.5).sum(),
    "Neutral > 0.7": (df_over["finbert_neutral"] > 0.7).sum(),
}

for k, v in summary.items():
    print(f"{k}: {v}")


총 개수: 8
평균 Positive: 0.30173408985865535
평균 Neutral: 0.6699639637954533
평균 Negative: 0.028301946480496512
Positive > 0.5: 2
Negative > 0.5: 0
Neutral > 0.7: 5


In [14]:
summary = {
    "총 개수": len(df),
    "평균 Positive": df["finbert_positive"].mean(),
    "평균 Neutral": df["finbert_neutral"].mean(),
    "평균 Negative": df["finbert_negative"].mean(),
    "Positive > 0.5": (df["finbert_positive"] > 0.5).sum(),
    "Negative > 0.5": (df["finbert_negative"] > 0.5).sum(),
    "Neutral > 0.7": (df["finbert_neutral"] > 0.7).sum(),
}

for k, v in summary.items():
    print(f"{k}: {v}")

총 개수: 8590
평균 Positive: 0.27216195756895806
평균 Neutral: 0.5354320729773412
평균 Negative: 0.19240596977245797
Positive > 0.5: 2322
Negative > 0.5: 1639
Neutral > 0.7: 4415
